# In Depth Tutorial on the MyUR3e Class #

## Introduction

The MyUR3e class is a part of the myur library and enables advanced control of the MyUR3e arm through simple Python commands. This tutorial provides a comprehensive guide to using the `MyUR3e` class for controlling the UR3e robot arm. The `MyUR3e` class offers various functionalities, including basic operations like reading joint states, controlling the gripper, and moving the arm.

We will walk through each feature with practical examples.


## Initialization ##

In [ ]:
# Import the MyUR3e class from the myur library
from myur import MyUR3e

# Create an instance of the MyUR3e class
robot = MyUR3e()

## Reading Data Fields ##
The MyUR3e class has data fields for joint angles, velocities, efforts, end effector position, force, torque, and gripper settings. The `read` functions give access to the most recent data point for the desired value.

In [ ]:
import time
from IPython.display import clear_output

while True:
    clear_output(wait=True)
    print(
        f"Joint Position: {robot.read_joints_pos()}"
    )  # List [Pan, Lift, Elbow, Wrist1, Wrist2, Wrist3] radians
    print(
        f"Joint Velocity: {robot.read_joints_vel()}"
    )  # List [Pan, Lift, Elbow, Wrist1, Wrist2, Wrist3] radians / s
    print(
        f"Joint Effort: {robot.read_joints_eff()}"
    )  # List [Pan, Lift, Elbow, Wrist1, Wrist2, Wrist3]
    print(f"Global Position: {robot.read_global_pos()}")  # List [x,y,z,rx,ry,rz] meters & degrees
    print(f"End Effector Force: {robot.read_force()}")  # List [x,y,z] # newtons
    print(f"End Effector Torque: {robot.read_torque()}")  # List [rx,ry,rz] # newton meters
    print(
        f"Gripper Settings: {robot.read_gripper()}"
    )  # List [Position, Speed, Force] (0-100)
    time.sleep(0.05)

## Movement
### Global & Joint Movement
You can move the robot arm using either global cartesian coordinates or joint angles. Both systems have advantages and disadvantages. Specifying global coordinates is certainly easier to use and will require less work. However, by only specifying the position of the end effector you are leaving the specific pose of the arm up to the IK solver (inverse kinematic solver). By default, the IK solver will find the pose closest to the arms current position. There are instances when this might not give you the desired pose and at that point you should specify the position using `move_joints`. Additionaly, using `move_joints` allows you to create isolated movements like panning the robot left or right or rotating the wrist.

In [ ]:
# Create a point using global coordinates
global_point = [[0.2, 0.3, 0.3, 0.0, 0.0, 0.0]] # [x,y,z,rx,ry,rz]

# Move the robot to a new pose using global end effector position
robot.move_global(global_point)

In [ ]:
# Create a point using joint positions
joints_point = [[0.304, -1.765, 1.762, -1.567, -1.572, 5.016]] # [pan,lift,elbow,wrist1,wrist2,wrist3]

# Move the robot to a new pose using joint angles
robot.move_joints(joints_point)

If you are using both `move_global` and `move_joints`, be careful when switching between them. Joints can move more than 360 degrees, and as a result specified joint angles might not be the closest rotation to the robots current pose. It is also possible to pass joint angles into `move_global` and vice versa. More often than not this will produce an IK error but it is possible a solution will exist and the arm will move to an undesired pose.

### Relative & Absolute Movement
When using `move_global` or `move_joints` you are specifying the absolute location you want the end effector or joints to move to. If you don't know or care exactly where the arm is however, it might useful to move the arm using relative commands. For example, if you were receiving directions to navigate a city an absolute command might be "walk to 33rd and Broadway" whereas a relative command would be "walk 4 blocks North and 2 blocks East." You can make relative movement commands using `move_global_r` and `move_joints_r`. With relative commands, a zero represents no movement in that axis. 

In [ ]:
# Move 10cm in the x axis using relative motion
relative_global = [[0.1, 0.0, 0.0, 0.0, 0.0, 0.0]]  # [x,y,z,rx,ry,rz]

# Move the robot using the relative command
# QUESTION: explore time with relative trajectories
robot.move_global_r(relative_global)

In [ ]:
# Move the pan joint 0.1 radian using relative motion
relative_joints = [[0.1, 0.0, 0.0, 0.0, 0.0, 0.0]]  # [pan,lift,elbow,wrist1,wrist2,wrist3]

# Move the robot using the relative command
# QUESTION: explore time with relative trajectories
robot.move_joints_r(relative_joints)

### The Gripper
The UR3e has a gripper attached to the end effector that can be used for picking up and grasping objects. You can open and close it using the `move_gripper` function.

In [ ]:
# Open the gripper
robot.move_gripper(0) # 0 = open
robot.move_gripper(100) # 100 = closed

For more fine control over the gripper, you can pass through speed and force parameters. The default for speed and force is 50.

In [ ]:
# Close the gripper
robot.move_gripper(100,50,50) # position=100, speed=50, force=50

### Trajectories
If you want the arm to move in a specific motion, you can do so with trajectories! Instead of passing through a single point, we will just pass through a list of many points that the arm will move to in a consecutive manner.

In [ ]:
# Create a two point trajectory using global coordinates
trajectory = [[0.2, 0.2, 0.3, 0.0, 0.0, 0.0],
              [0.3, 0.3, 0.3, 0.0, 0.0, 0.0]]

# Move the end effector through the trajectory
robot.move_global(trajectory)

You can do the same thing with relative commands. Each "point" in a relative trajectory is based on the point before it.

In [ ]:
# Move in a 10cm square using global relative commands
relative_trajectory = [
    [-0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, -0.1, 0.0, 0.0, 0.0, 0.0],
]

# Move the end effector through the trajectory
robot.move_global_r(trajectory)

### Time
The default duration of a given trajectory is 5 seconds. Additionally, out of precaution the arm will take 5 seconds to move to the first pose of a given trajectory. The `time` parameter allows you to alter both of these conditions, however it is important that you do this very intentionally as an incorrectly specified time parameter can result in dangerously fast arm movement.

In [ ]:
# Take 3 seconds to arrive at the trajectory and then again to complete it
robot.move_global(trajectory, time=3)

# Take 3 seconds to arrive at the trajectory and then 5 to complete it
robot.move_global(trajectory, time=(3,5))

### Interpolation
If you look very closely, you might notice that the arm's end effector didn't actually go in a straight line from the first point to the second point. This is because the arm found the most efficient path from A to B, which doesn't always result in linear movement. We can fix this with the `interp` parameter.

In [ ]:
# Interpolate the trajectory between given points
robot.move_global(trajectory,time=(3,5),interp="linear")

In addition to linear interpolation, there are also options for arc and spline interpolations (think first order, second order, third order polynomials). As a result the linear interpolation requires a minimum trajectory of two points, the arc requires three, and the spline requires four. Lets create a more interesting trajectory to try this out!

In [ ]:
zigzag = [
    [0.2, 0.25, 0.3, 0, 0, 0],
    [0.2, 0.3, 0.3, 0, 0, 0],
    [0.1, 0.25, 0.3, 0, 0, 0],
    [0.2, 0.25, 0.3, 0, 0, 0],
]

# Interpolate the points with a first order equation
robot.move_global(zigzag, time_step=(3, 5),interp="linear")

In [ ]:
# Interpolate the points with a second order equation
robot.move_global(zigzag, time_step=(3, 5), interp="arc")

In [ ]:
# Interpolate the points with a third order equation
robot.move_global(zigzag, time_step=(3, 5), interp="spline")

## Trajectory Features
### Trajectory Visualization
The difference between each interpolation method might be hard to see as the robot moves. Lets try visualizing the trajectories instead! Whenever you run a trajectory using a move command the points will automatically be added to a 3D plot stored in a local html file. You can open the HTML file in your Jupyter Notebook with the code below or by opening the file in your browser.

In [ ]:
from IPython.display import HTML

# Open HTML file as a cell output in your Jupyter Notebook
HTML(filename="ur3e_trajectory.html")

When debugging your program you might want to add a trajectory to the visualization without moving the robot, or clear all trajectories from the plot to start over. You can do so with the `clear_vis` function and the `vis_only` parameter.

In [ ]:
# Clear all trajectories from the plot
robot.clear_vis()

# Add a trajectory to the plot without moving the robot
robot.move_global(zigzag, time_step=(3, 5), interp="spline", vis_only=True)

When viewing a plot, colored points represent end effector locations and grey segments represent the orientation of the end effector. To hide a trajectory or orientation click on the label in the legend. To isolate a trajectory or orientation double click the label in the legend. To get more information about a trajectory point, hover your mouse over the point. If your trajectories are returning IK errors, the unsolved points will appear as larger red markers in the plot.

### Saving Trajectories
When programming the robot, you might create a trajectory that you want to save in your code without defining the lengthy coordinates in visible text. The `save_trajectory` and `get_trajectory` functions will allow you to label, store, and access trajectories in a local .json file.

In [ ]:
# Save the trajectory zigzag with the name "interpolation_test"
robot.save_trajectory("interpolation_test",zigzag)

# Access the saved trajectory named "interpolation_test"
interpolation_test = robot.get_trajectory("interpolation_test")

You can also run your saved trajectories directly inside of a move function.

In [ ]:
# Move the robot along the interpolation test trajectory using spline interpolation
robot.move("interpolation_test", time_step=(3, 5), interp="spline")

### Recording Trajectories
Some trajectories are harder to parametrically define than others, and if you don't care much about precision then you might want to just move the robot yourself and ask it to mirror you later! This is also useful when it is important *how* the robot moves from point A to point B. If you only specify A and B, the robot might not do what you want in between. If you use the `record` function however you can take complete control of the robot's motion. This is a great solution to transitional trajectories that are moving your end effector from one active location to another.

In [ ]:
# Record a trajectory called my_motion
robot.record("my_motion")

To use the record function, pause the UR3e's external control program using the Teach Pendant. Run the record function and put the UR3e into FreeDrive. The recording will start as soon as the arm moves and end once it comes to a stop. The default function will record a point every second. You can vary this using the `interval` parameter as well as change the threshold for movement detection. A large interval will result in smoother motion when paired with spline interpolation.

In [ ]:
# This will record fewer points (every 2 seconds)
robot.record("my_motion2",interval=2,threshold=0.001)

## Non Blocking Commands